In [10]:
from hello.hello3 import HelloApp, NotLoggedInError
import datetime
from hello import hello_logger
import os
import requests
import time

In [11]:
# General functions

def call(f, *a, _quiet=False, **k):
    if not _quiet:
        now = datetime.datetime.now().strftime("%H:%M:%S")
        s = "(" + ", ".join(map(repr, a)) + ")"
        print("%s Calling %s%s" % (now, f.__name__, s))
    while True:
        try:
            return f(*a, **k)
        except NotLoggedInError:
            h.login()
        except requests.exceptions.Timeout as e:
            print("Got timeout! Make sure you're on the right network!")
        except (BadError, IOError, requests.exceptions.ConnectionError) as e:
            print("Error occurred:", e.__class__)   
        time.sleep(1)
            
def connect(ip, *args):
    global h
    h = HelloApp(ip, *args)
    
def dfmt(d):
    return d.strftime("%m/%d/%Y %I:%M:%S %p")
    
def wait1(s):
    if s <= 0: return
    dend = dfmt(datetime.datetime.now() + datetime.timedelta(seconds=s))
    print("Sleeping %d seconds, ending at %s" %(s,dend))
    start = time.time()
    end = start + s
    while time.time() < end:
        time.sleep(5)
    
def wait3(s, src):
    start = datetime.datetime.now()
    end = start + datetime.timedelta(seconds=s)
    m1 = dfmt(start)
    m2 = dfmt(end)
    print("\r(%s) Sleeping %d seconds from %s to %s          " % (src, s, m1, m2), end="")
    tend = time.time() + s
    while time.time() < tend:
        time.sleep(5)
    
def wait2(s, *a, **kw):
    pass
wait = wait1

### Global functions and routines

In [12]:
# Specific functions

# globals
logger = None
_mv = None

def setconfig(g, s, v):
    call(h.setconfig, g, s, v)

def getmv():
    global _mv
    _mv = call(h.gpmv, _quiet=True)
    return _mv
    
def setup_controls(rpm=18):
    setag(rpm)
    call(h.setmg, 2, 0)
    
def setag(rpm):
    call(h.setag, 0, rpm)
    
def settemp(t):
    while True:
        call(h.settemp, 0, t)
        mv = getmv()
        tm = mv['temperature']
        if tm['sp'] == t and tm['mode'] == 0:
            break
        else:
            print("Retrying call, failed to set temp")
    
def tempoff():
    call(h.settemp, 2, getmv()['temperature']['sp'])
    
def endbatch():
    if call(h.batchrunning):
        call(h.endbatch)
    
def startbatch(n):
    endbatch()
    call(h.startbatch, n)
    
def temppv():
    return getmv()['temperature']['pv']
    
def waittemp_lower(t, timeout=8*3600):
    waited = 0
    while temppv() > t and (timeout<=0 or waited<timeout):
        wait3(60, 'waittemp_lower')
        waited += 60
    print()
    
def waittemp_higher(t, timeout=8*3600):
    waited = 0
    while temppv() < t and (timeout<=0 or waited<timeout):
        wait3(60, 'waittemp_higher')
        waited += 60
    print()  

    
def rampup(sp, pm=0.05, timeout=4*3600):
    settemp(sp)
    waittemp_higher(sp-pm, timeout)
    
def rampdown(sp, pm=0.0, timeout=10*3600):
    tempoff()
    waittemp_lower(sp+pm, timeout)
    
def newbatch(i, j):
    startbatch(batch_tmpl%(i,j))
    
def mkpid(p, i):
    def pid():
        call(h.setconfig, "Temperature", "P_Gain_(%/C)", p)
        call(h.setconfig, "Temperature", 'I_Time_(min)', i)
    return pid
        

In [38]:
_bni = 1
def nb(pref, i=1, n=1):
    bl = call(h.getBatches)
    b = bl.names_to_batches
    s = "%s %d.%d" % (pref, i, n)
    while s in b:
        n += 1
        s = "%s %d.%d" % (pref, i, n)
    startbatch(s)

def tempoff():
    settemp(0)
    
def setbn(i):
    global _bni
    _bni = i

def o2a():
    nb("tpid", _bni)
    settemp(37)
    wait(3*hours)
    endbatch()
    
def cool(hrs=8):
    tempoff()
    wait(hrs*hours)
    
def a2a(end):
    nb("tpid", _bni)
    settemp(end)
    wait(3*hours)
    endbatch()

def verify(pid):
    print("Verifying settings")
    pid()
    o2a()
    a2a(35)
    a2a(37)
    
def verify2(pid):
    print("Verifying settings")
    pid()
    o2a()
 
def tpv():
    return call(h.gpmv)['temperature']['pv']

def waitcool(sp, t=3600):
    tempoff()
    while tpv() >= sp:
        print("waitcool")
        wait(t)
        
def runrecipe(recipe):
    call(h.login)
    return call(h.call, "runrecipe", recipe=recipe)

In [39]:
def setup_logger(interval=10):
    global logger
    keys = [
        'temperature.pv',
        'temperature.output',
        'temperature.sp',
        'agitation.pv',
        'level.pv',
    ]
    if logger is not None:
        logger.stop()
    logger = hello_logger.HelloLogger(h, interval, None, keys)
    logger.start()
    
def logfile(fn):
    print("Creating new file: %r"%fn)
    logger.new_file(fn)
    
def endlog():
    logger.new_file(None)
def ntests(n):
    return range(1,n+1)

In [40]:
fn_tmpl = "Heating test %s %s.csv"

def new_file(name):
    logfile(fn_tmpl% (name, bt()))

def bt(): 
    return datetime.datetime.now().strftime("%y%m%d")

def mkfldr(p):
    path = "C:\\Users\\natha\\Documents\\test\\"+p
    os.makedirs(path, exist_ok=True)
    os.chdir(path)
    
def chd(p):
    os.chdir("C:\\Users\\natha\\Documents\\test\\"+p)

In [49]:
connect('192.168.1.7')
setup_logger()
wait(60)
mkfldr("Intellia 3L Mag Rapid Heat Test")
new_file("Man Recipe 100pc off 33")
runrecipe("intelliaheat")

Sleeping 60 seconds, ending at 12/15/2017 02:43:00 PM
Creating new file: 'Heating test Man Recipe 100pc off 33 171215.csv'
14:43:00 Calling login()
14:43:00 Calling call('runrecipe')


<Response [200]>

In [1]:
#logger.write_to_file()

In [46]:
logger.stop()